In [1]:
# Import necessary modules
import sys
sys.path.append('../fraud_detection')  # Adjust this path based on your working directory
from utils.logging_util import setup_logger

# notebooks/eda_fraud_detection.ipynb

# Import the necessary functions from your project
# from model_training import prepare_data, train_logistic_regression , train_random_forest , train_gradient_boosting ,train_decision_tree,  train_mlp,train_rnn,train_cnn,train_lstm
from model_training import (prepare_data, prepare_credit_card_data,train_logistic_regression, train_random_forest, 
                            train_gradient_boosting, train_decision_tree, )

from model_evaluation import evaluate_model

from model_explainability import explain_model_shap, explain_model_lime


c:\Users\Akram 1\Desktop\New folder (2)\fraud-detection-for-e-commercand-bank-transactions\myvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_path = '../data/fraud_datas.csv'
prepared_data = prepare_data(data_path, target_column='fraud_class')

2024-11-20 13:08:05,994 - root - INFO - Loaded data from ../data/fraud_datas.csv
2024-11-20 13:08:05,994 - root - INFO - Loaded data from ../data/fraud_datas.csv
2024-11-20 13:08:05,994 - root - INFO - Loaded data from ../data/fraud_datas.csv
2024-11-20 13:08:06,025 - root - INFO - Encoded column: device_id
2024-11-20 13:08:06,025 - root - INFO - Encoded column: device_id
2024-11-20 13:08:06,025 - root - INFO - Encoded column: device_id
2024-11-20 13:08:06,042 - root - INFO - Encoded column: source
2024-11-20 13:08:06,042 - root - INFO - Encoded column: source
2024-11-20 13:08:06,042 - root - INFO - Encoded column: source
2024-11-20 13:08:06,043 - root - INFO - Encoded column: browser
2024-11-20 13:08:06,043 - root - INFO - Encoded column: browser
2024-11-20 13:08:06,043 - root - INFO - Encoded column: browser
2024-11-20 13:08:06,043 - root - INFO - Encoded column: sex
2024-11-20 13:08:06,043 - root - INFO - Encoded column: sex
2024-11-20 13:08:06,043 - root - INFO - Encoded column: se

In [4]:
# Path to the credit card dataset
data_path = "../data/credit_datas.csv"
target_column = "Class"  # Target column indicating fraud (1 = Fraud, 0 = Non-Fraud)

# Prepare the data for credit card fraud detection
credit_data = prepare_credit_card_data(data_path, target_column)

2024-11-20 13:08:16,038 - root - INFO - Loaded data from ../data/credit_datas.csv
2024-11-20 13:08:16,038 - root - INFO - Loaded data from ../data/credit_datas.csv
2024-11-20 13:08:16,038 - root - INFO - Loaded data from ../data/credit_datas.csv
2024-11-20 13:08:16,288 - root - INFO - Data successfully split into training and testing sets.
2024-11-20 13:08:16,288 - root - INFO - Data successfully split into training and testing sets.
2024-11-20 13:08:16,288 - root - INFO - Data successfully split into training and testing sets.
2024-11-20 13:08:16,516 - root - INFO - Feature scaling applied.
2024-11-20 13:08:16,516 - root - INFO - Feature scaling applied.
2024-11-20 13:08:16,516 - root - INFO - Feature scaling applied.


In [5]:


# Accessing the training and testing sets for credit card fraud detection
X_train_credit = credit_data['X_train']
X_test_credit = credit_data['X_test']
y_train_credit = credit_data['y_train']
y_test_credit = credit_data['y_test']
scaler_credit = prepared_data.get('scaler')
# Display confirmation
print("Credit card fraud data successfully prepared.")
print(f"Training set size: {X_train_credit.shape[0]} samples")
print(f"Test set size: {X_test_credit.shape[0]} samples")


Credit card fraud data successfully prepared.
Training set size: 226980 samples
Test set size: 56746 samples


In [ ]:

# Access the prepared data
X_train = prepared_data['X_train']
X_test = prepared_data['X_test']
y_train = prepared_data['y_train']
y_test = prepared_data['y_test']
scaler = prepared_data.get('scaler')

# Display confirmation
print("fraud data successfully prepared.")
print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")

In [ ]:

# Access the prepared data
X_train = prepared_data['X_train']
X_test = prepared_data['X_test']
y_train = prepared_data['y_train']
y_test = prepared_data['y_test']
scaler = prepared_data.get('scaler')

# Display confirmation
print("Credit card fraud data successfully prepared.")
print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")

# Initialize and fit the scaler

# Train and save the Logistic Regression model and its scaler
logistic_model_path = '../models/logistic_regression_model.pkl'
logistic_scaler_path = '../models/logistic_regression_scaler.pkl'
logistic_results = train_logistic_regression(X_train, X_test, y_train, y_test, logistic_model_path, scaler, logistic_scaler_path)

# Train and save the Random Forest model and its scaler
random_forest_model_path = '../models/random_forest_model.pkl'
random_forest_scaler_path = '../models/random_forest_scaler.pkl'
random_forest_results = train_random_forest(X_train, X_test, y_train, y_test, random_forest_model_path, scaler, random_forest_scaler_path)

# Display results
print(logistic_results)
print(random_forest_results)


2024-11-20 13:08:28,167 - root - INFO - Training model: LogisticRegression.
2024-11-20 13:08:28,167 - root - INFO - Training model: LogisticRegression.
2024-11-20 13:08:28,167 - root - INFO - Training model: LogisticRegression.
2024-11-20 13:08:28,292 - root - INFO - Model saved to ../models/logistic_regression_model.pkl.
2024-11-20 13:08:28,292 - root - INFO - Model saved to ../models/logistic_regression_model.pkl.
2024-11-20 13:08:28,292 - root - INFO - Model saved to ../models/logistic_regression_model.pkl.
2024-11-20 13:08:28,292 - root - INFO - Scaler saved to ../models/logistic_regression_scaler.pkl.
2024-11-20 13:08:28,292 - root - INFO - Scaler saved to ../models/logistic_regression_scaler.pkl.
2024-11-20 13:08:28,292 - root - INFO - Scaler saved to ../models/logistic_regression_scaler.pkl.
2024-11-20 13:08:28,343 - root - INFO - Model training and evaluation completed successfully.
2024-11-20 13:08:28,343 - root - INFO - Model training and evaluation completed successfully.
20

Credit card fraud data successfully prepared.
Training set size: 103316 samples
Test set size: 25830 samples


2024-11-20 13:09:02,681 - root - INFO - Model saved to ../models/random_forest_model.pkl.
2024-11-20 13:09:02,681 - root - INFO - Model saved to ../models/random_forest_model.pkl.
2024-11-20 13:09:02,681 - root - INFO - Model saved to ../models/random_forest_model.pkl.
2024-11-20 13:09:02,681 - root - INFO - Scaler saved to ../models/random_forest_scaler.pkl.
2024-11-20 13:09:02,681 - root - INFO - Scaler saved to ../models/random_forest_scaler.pkl.
2024-11-20 13:09:02,681 - root - INFO - Scaler saved to ../models/random_forest_scaler.pkl.
2024-11-20 13:09:04,068 - root - INFO - Model training and evaluation completed successfully.
2024-11-20 13:09:04,068 - root - INFO - Model training and evaluation completed successfully.
2024-11-20 13:09:04,068 - root - INFO - Model training and evaluation completed successfully.


{'model': LogisticRegression(class_weight='balanced', max_iter=2000), 'training_time': 0.12279391288757324, 'roc_auc': np.float64(0.7624917395087755), 'report': '              precision    recall  f1-score   support\n\n           0       0.95      0.64      0.76     23389\n           1       0.17      0.71      0.27      2441\n\n    accuracy                           0.64     25830\n   macro avg       0.56      0.67      0.52     25830\nweighted avg       0.88      0.64      0.72     25830\n'}
{'model': RandomForestClassifier(class_weight='balanced'), 'training_time': 34.18590998649597, 'roc_auc': np.float64(0.7789021908970993), 'report': '              precision    recall  f1-score   support\n\n           0       0.95      1.00      0.98     23389\n           1       1.00      0.55      0.71      2441\n\n    accuracy                           0.96     25830\n   macro avg       0.98      0.77      0.84     25830\nweighted avg       0.96      0.96      0.95     25830\n'}


In [7]:
# Access the prepared credit card fraud data
X_train_credit = credit_data['X_train']
X_test_credit = credit_data['X_test']
y_train_credit = credit_data['y_train']
y_test_credit = credit_data['y_test']
credit_scaler = credit_data.get('scaler')

# Train and save the Logistic Regression model for credit card fraud detection
credit_logistic_model_path = '../models/credit_logistic_regression_model.pkl'
credit_logistic_scaler_path = '../models/credit_logistic_scaler.pkl'
credit_logistic_results = train_logistic_regression(
    X_train_credit, X_test_credit, y_train_credit, y_test_credit, 
    credit_logistic_model_path, credit_scaler, credit_logistic_scaler_path
)

# Train and save the Random Forest model for credit card fraud detection
credit_random_forest_model_path = '../models/credit_random_forest_model.pkl'
credit_random_forest_scaler_path = '../models/credit_random_forest_scaler.pkl'
credit_random_forest_results = train_random_forest(
    X_train_credit, X_test_credit, y_train_credit, y_test_credit, 
    credit_random_forest_model_path, credit_scaler, credit_random_forest_scaler_path
)

# Display results for credit card fraud detection models
print("\nLogistic Regression Results (Credit Card Fraud):")
print(credit_logistic_results)

print("\nRandom Forest Results (Credit Card Fraud):")
print(credit_random_forest_results)

2024-11-20 15:06:42,256 - root - INFO - Training model: LogisticRegression.
2024-11-20 15:06:42,256 - root - INFO - Training model: LogisticRegression.
2024-11-20 15:06:42,256 - root - INFO - Training model: LogisticRegression.
2024-11-20 15:06:44,144 - root - INFO - Model saved to ../models/credit_logistic_regression_model.pkl.
2024-11-20 15:06:44,144 - root - INFO - Model saved to ../models/credit_logistic_regression_model.pkl.
2024-11-20 15:06:44,144 - root - INFO - Model saved to ../models/credit_logistic_regression_model.pkl.
2024-11-20 15:06:44,144 - root - INFO - Scaler saved to ../models/credit_logistic_scaler.pkl.
2024-11-20 15:06:44,144 - root - INFO - Scaler saved to ../models/credit_logistic_scaler.pkl.
2024-11-20 15:06:44,144 - root - INFO - Scaler saved to ../models/credit_logistic_scaler.pkl.
2024-11-20 15:06:44,241 - root - INFO - Model training and evaluation completed successfully.
2024-11-20 15:06:44,241 - root - INFO - Model training and evaluation completed success


Logistic Regression Results (Credit Card Fraud):
{'model': LogisticRegression(class_weight='balanced', max_iter=2000), 'training_time': 1.8838436603546143, 'roc_auc': np.float64(0.9754564584706141), 'report': '              precision    recall  f1-score   support\n\n           0       1.00      0.98      0.99     56656\n           1       0.06      0.89      0.11        90\n\n    accuracy                           0.98     56746\n   macro avg       0.53      0.93      0.55     56746\nweighted avg       1.00      0.98      0.99     56746\n'}

Random Forest Results (Credit Card Fraud):
{'model': RandomForestClassifier(class_weight='balanced'), 'training_time': 191.6191279888153, 'roc_auc': np.float64(0.9370552103925445), 'report': '              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00     56656\n           1       0.98      0.71      0.83        90\n\n    accuracy                           1.00     56746\n   macro avg       0.99      0.86  

In [ ]:
# Define paths for Decision Tree and Gradient Boosting models
decision_tree_model_path = '../models/decision_tree_model.pkl'
gradient_boosting_model_path = '../models/gradient_boosting_model.pkl'

# Train and save the Decision Tree model
decision_tree_results = train_decision_tree(
    X_train, X_test, y_train, y_test, decision_tree_model_path
)

# Train and save the Gradient Boosting model
gradient_boosting_results = train_gradient_boosting(
    X_train, X_test, y_train, y_test, gradient_boosting_model_path
)


print("\nDecision Tree Model Results:")
print(decision_tree_results)

print("\nGradient Boosting Model Results:")
print(gradient_boosting_results)


In [ ]:

# Define paths for Decision Tree and Gradient Boosting models
credit_decision_tree_model_path = '../models/credit_decision_tree_model.pkl'
credit_gradient_boosting_model_path = '../models/credit_gradient_boosting_model.pkl'

# Train and save the Decision Tree model for credit card fraud detection
credit_decision_tree_results = train_decision_tree(
    X_train_credit, X_test_credit, y_train_credit, y_test_credit, credit_decision_tree_model_path
)

# Train and save the Gradient Boosting model for credit card fraud detection
credit_gradient_boosting_results = train_gradient_boosting(
    X_train_credit, X_test_credit, y_train_credit, y_test_credit, credit_gradient_boosting_model_path
)


print("\nDecision Tree Model Results (Credit Card Fraud):")
print(credit_decision_tree_results)

print("\nGradient Boosting Model Results (Credit Card Fraud):")
print(credit_gradient_boosting_results)


In [ ]:
# Extract the model objects from the training results
logistic_model = logistic_results["model"]
decision_tree_model = decision_tree_results["model"]
random_forest_model = random_forest_results["model"]
gradient_boosting_model = gradient_boosting_results["model"]

# Evaluate each model
logistic_evaluation = evaluate_model(logistic_model, X_test, y_test)
decision_tree_evaluation = evaluate_model(decision_tree_model, X_test, y_test)
random_forest_evaluation = evaluate_model(random_forest_model, X_test, y_test)
gradient_boosting_evaluation = evaluate_model(gradient_boosting_model, X_test, y_test)

# Display evaluation results
print("Logistic Regression Evaluation:")
print(logistic_evaluation)

print("\nDecision Tree Evaluation:")
print(decision_tree_evaluation)

print("\nRandom Forest Evaluation:")
print(random_forest_evaluation)

print("\nGradient Boosting Evaluation:")
print(gradient_boosting_evaluation)

In [ ]:
# Extract the model objects from the training results for credit card fraud detection
credit_logistic_model = credit_logistic_results["model"]
credit_decision_tree_model = credit_decision_tree_results["model"]
credit_random_forest_model = credit_random_forest_results["model"]
credit_gradient_boosting_model = credit_gradient_boosting_results["model"]

# Evaluate each credit card fraud detection model
credit_logistic_evaluation = evaluate_model(credit_logistic_model, X_test_credit, y_test_credit)
credit_decision_tree_evaluation = evaluate_model(credit_decision_tree_model, X_test_credit, y_test_credit)
credit_random_forest_evaluation = evaluate_model(credit_random_forest_model, X_test_credit, y_test_credit)
credit_gradient_boosting_evaluation = evaluate_model(credit_gradient_boosting_model, X_test_credit, y_test_credit)

# Display evaluation results for credit card fraud detection models
print("Logistic Regression Evaluation (Credit Card Fraud):")
print(credit_logistic_evaluation)

print("\nDecision Tree Evaluation (Credit Card Fraud):")
print(credit_decision_tree_evaluation)

print("\nRandom Forest Evaluation (Credit Card Fraud):")
print(credit_random_forest_evaluation)

print("\nGradient Boosting Evaluation (Credit Card Fraud):")
print(credit_gradient_boosting_evaluation)


In [ ]:
# Explaining Logistic Regression Model
print("Explaining Logistic Regression Model:")

# SHAP explanation for Logistic Regression
logistic_shap_values = explain_model_shap(logistic_model, X_train, X_test, model_type='lr')

# LIME explanation for Logistic Regression
print("LIME explanation for Logistic Regression:")
explanation_lime_lr = explain_model_lime(logistic_model, X_train, X_test, idx=0)


# Explaining Decision Tree Model
print("Explaining Decision Tree Model:")

# SHAP explanation for Decision Tree
decision_tree_shap_values = explain_model_shap(decision_tree_model, X_train, X_test, model_type='rf')

# LIME explanation for Decision Tree
print("LIME explanation for Decision Tree:")
explanation_lime_dt = explain_model_lime(decision_tree_model, X_train, X_test, idx=0)


# Explaining Random Forest Model
print("Explaining Random Forest Model:")

# SHAP explanation for Random Forest
random_forest_shap_values = explain_model_shap(random_forest_model, X_train, X_test, model_type='rf')

# LIME explanation for Random Forest
print("LIME explanation for Random Forest:")
explanation_lime_rf = explain_model_lime(random_forest_model, X_train, X_test, idx=0)


# Explaining Gradient Boosting Model
print("Explaining Gradient Boosting Model:")

# SHAP explanation for Gradient Boosting
gradient_boosting_shap_values = explain_model_shap(gradient_boosting_model, X_train, X_test, model_type='rf')

# LIME explanation for Gradient Boosting
print("LIME explanation for Gradient Boosting:")
explanation_lime_gb = explain_model_lime(gradient_boosting_model, X_train, X_test, idx=0)


In [ ]:
print(X_train.shape)  # Output will be in the form (num_samples, num_features)


In [ ]:
# Explaining Logistic Regression Model (Credit Card Fraud)
print("Explaining Logistic Regression Model (Credit Card Fraud):")

# SHAP explanation for Logistic Regression
credit_logistic_shap_values = explain_model_shap(
    credit_logistic_model, X_train_credit, X_test_credit, model_type='lr'
)

# LIME explanation for Logistic Regression
print("LIME Explanation for Logistic Regression (Credit Card Fraud):")
credit_explanation_lime_lr = explain_model_lime(
    credit_logistic_model, X_train_credit, X_test_credit, idx=0
)


# Explaining Decision Tree Model (Credit Card Fraud)
print("Explaining Decision Tree Model (Credit Card Fraud):")

# SHAP explanation for Decision Tree
credit_decision_tree_shap_values = explain_model_shap(
    credit_decision_tree_model, X_train_credit, X_test_credit, model_type='rf'
)

# LIME explanation for Decision Tree
print("LIME Explanation for Decision Tree (Credit Card Fraud):")
credit_explanation_lime_dt = explain_model_lime(
    credit_decision_tree_model, X_train_credit, X_test_credit, idx=0
)


# Explaining Random Forest Model (Credit Card Fraud)
print("Explaining Random Forest Model (Credit Card Fraud):")

# SHAP explanation for Random Forest
credit_random_forest_shap_values = explain_model_shap(
    credit_random_forest_model, X_train_credit, X_test_credit, model_type='rf'
)

# LIME explanation for Random Forest
print("LIME Explanation for Random Forest (Credit Card Fraud):")
credit_explanation_lime_rf = explain_model_lime(
    credit_random_forest_model, X_train_credit, X_test_credit, idx=0
)


# Explaining Gradient Boosting Model (Credit Card Fraud)
print("Explaining Gradient Boosting Model (Credit Card Fraud):")

# SHAP explanation for Gradient Boosting
credit_gradient_boosting_shap_values = explain_model_shap(
    credit_gradient_boosting_model, X_train_credit, X_test_credit, model_type='rf'
)

# LIME explanation for Gradient Boosting
print("LIME Explanation for Gradient Boosting (Credit Card Fraud):")
credit_explanation_lime_gb = explain_model_lime(
    credit_gradient_boosting_model, X_train_credit, X_test_credit, idx=0
)


In [ ]:

# # Define file paths for saving models
# mlp_model_path = '../models/mlp_model.h5'
# rnn_model_path = '../models/rnn_model.h5'
# cnn_model_path = '../models/cnn_model.h5'
# lstm_model_path = '../models/lstm_model.h5'

# # Train and save the MLP model
# print("Training MLP model...")
# mlp_model, mlp_history = train_mlp(X_train, X_test, y_train, y_test, mlp_model_path)
# print("\nMLP Model Training History:")
# print(mlp_history)



# # Train and save the LSTM model
# print("\nTraining LSTM model...")
# lstm_model, lstm_history = train_lstm(X_train, X_test, y_train, y_test, lstm_model_path)
# print("\nLSTM Model Training History:")
# print(lstm_history)

In [ ]:
# # Train and save the CNN model
# print("\nTraining CNN model...")
# cnn_model, cnn_history = train_cnn(X_train, X_test, y_train, y_test, cnn_model_path)
# print("\nCNN Model Training History:")
# print(cnn_history)

# # Train and save the RNN model
# print("\nTraining RNN model...")
# rnn_model, rnn_history = train_rnn(X_train, X_test, y_train, y_test, rnn_model_path)
# print("\nRNN Model Training History:")
# print(rnn_history)

In [ ]:
# # Evaluate the MLP, RNN, and LSTM models
# mlp_evaluation = evaluate_model(mlp_model, X_test, y_test)
# rnn_evaluation = evaluate_model(rnn_model, X_test, y_test)
# lstm_evaluation = evaluate_model(lstm_model, X_test, y_test)

# # Display evaluation results
# print("\nMLP Model Evaluation:")
# print(mlp_evaluation)

# print("\nRNN Model Evaluation:")
# print(rnn_evaluation)

# print("\nLSTM Model Evaluation:")
# print(lstm_evaluation)


In [ ]:
# # Paths to the saved models
# mlp_model_path = '../models/mlp_model.h5'
# rnn_model_path = '../models/rnn_model.h5'
# cnn_model_path = '../models/cnn_model.h5'
# lstm_model_path = '../models/lstm_model.h5'

# # Load the saved models
# mlp_model = load_model(mlp_model_path)
# rnn_model = load_model(rnn_model_path)
# cnn_model = load_model(cnn_model_path)
# lstm_model = load_model(lstm_model_path)

# # Test the models and evaluate them
# print("Evaluating MLP model...")
# mlp_evaluation = evaluate_model(mlp_model, X_test, y_test)
# print("MLP Model Evaluation:", mlp_evaluation)

# print("\nEvaluating RNN model...")
# rnn_evaluation = evaluate_model(rnn_model, X_test, y_test)
# print("RNN Model Evaluation:", rnn_evaluation)

# print("\nEvaluating CNN model...")
# cnn_evaluation = evaluate_model(cnn_model, X_test, y_test)
# print("CNN Model Evaluation:", cnn_evaluation)

# print("\nEvaluating LSTM model...")
# lstm_evaluation = evaluate_model(lstm_model, X_test, y_test)
# print("LSTM Model Evaluation:", lstm_evaluation)